In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/blackmamba-ops/Tomato_vision/blob/main/CNN_Project.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/blackmamba-ops/Emotion-Detection-Master.git">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>


# Importing necessary libraries

In [14]:
! pip install -q tensorboard

In [15]:
import numpy as np
import mediapipe as mp
import argparse
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [16]:
mode = "train"  # Change this to "display" if you want to display emotions

In [17]:
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

# data generators:

In [18]:
# Define data generators
train_dir = r'C:\Users\BASIL K AJI\Desktop\Projects\Emotion-detection-master\data\train'
val_dir = r'C:\Users\BASIL K AJI\Desktop\Projects\Emotion-detection-master\data\test'

num_train = 28709
num_val = 7178
batch_size = 64
num_epoch = 100

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


# model architecture:

In [19]:
# Create the model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))


c:\Users\BASIL K AJI\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


# Model training and display emotions

In [20]:
if mode == "train":
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

    tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1)

    model_info = model.fit(
            train_generator,
            steps_per_epoch=num_train // batch_size,
            epochs=num_epoch,
            validation_data=validation_generator,
            validation_steps=num_val // batch_size,
            callbacks=[tensorboard_callback])
    
    model.save('model.h5')

# Emotions will be displayed on your face from the webcam feed
elif mode == "display":
    model = load_model('model.h5')

    # Initialize the webcam
    cap = cv2.VideoCapture(0)

    # Load the Haarcascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Define the emotion labels
    emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

    # Start capturing frames
    with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Convert the frame to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Detect faces in the grayscale frame
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

            # Process each detected face
            for (x, y, w, h) in faces:
                # Detect landmarks in the face
                results = face_mesh.process(frame)

                # Draw landmarks on the frame
                if results.multi_face_landmarks:
                    for landmarks in results.multi_face_landmarks:
                        for landmark in landmarks.landmark:
                            cx, cy = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                            cv2.circle(frame, (cx, cy), 2, (0, 255, 0), cv2.FILLED)

                # Crop the face region
                face_roi = gray[y:y+h, x:x+w]

                # Resize the face region to 48x48 pixels (required input size for the model)
                face_roi_resized = cv2.resize(face_roi, (48, 48))

                # Preprocess the face region for prediction
                face_roi_resized = np.expand_dims(np.expand_dims(face_roi_resized, -1), 0) / 255.0

                # Predict emotion
                prediction = model.predict(face_roi_resized)
                max_index = int(np.argmax(prediction))
                emotion = emotion_dict[max_index]

                # Draw text showing the predicted emotion on the frame
                cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2, cv2.LINE_AA)

            # Display the frame with predicted emotion and landmarks
            cv2.imshow('Emotion Detection with MediaPipe Landmarks', frame)

            # Check for 'q' key press to exit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Release the webcam and close any open windows
        cap.release()
        cv2.destroyAllWindows()


Epoch 1/100


c:\Users\BASIL K AJI\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


448/448 ━━━━━━━━━━━━━━━━━━━━ 370s 806ms/step - accuracy: 0.2485 - loss: 1.8287 - val_accuracy: 0.3516 - val_loss: 1.6738
Epoch 2/100
  1/448 ━━━━━━━━━━━━━━━━━━━━ 1:56 261ms/step - accuracy: 0.2188 - loss: 1.6926

c:\Users\BASIL K AJI\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


448/448 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.2188 - loss: 0.8482 - val_accuracy: 0.4000 - val_loss: 0.8535
Epoch 3/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 145s 322ms/step - accuracy: 0.3569 - loss: 1.6434 - val_accuracy: 0.4141 - val_loss: 1.5348
Epoch 4/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 1s 943us/step - accuracy: 0.3750 - loss: 0.7818 - val_accuracy: 0.3000 - val_loss: 0.7578
Epoch 5/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 122s 272ms/step - accuracy: 0.4040 - loss: 1.5466 - val_accuracy: 0.4410 - val_loss: 1.4656
Epoch 6/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4844 - loss: 0.7246 - val_accuracy: 0.4000 - val_loss: 0.6780
Epoch 7/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 125s 278ms/step - accuracy: 0.4378 - loss: 1.4688 - val_accuracy: 0.4717 - val_loss: 1.3982
Epoch 8/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4219 - loss: 0.7606 - val_accuracy: 0.5000 - val_loss: 0.7490
Epoch 9/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 139s 310ms/step - accuracy: 0.4627 - loss: 1.4043 - va

In [21]:
%load_ext tensorboard
%tensorboard --logdir=./logs


Launching TensorBoard...